In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
base_url = "https://biobank.ndph.ox.ac.uk/ukb/"
category_url = "https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=100026"

In [3]:
response = requests.get(category_url)
response.raise_for_status()

In [4]:
# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table - usually it's the first table on the page
tables = soup.find_all('table')

# Initialize list to store data
category_data = []

In [7]:
# Look for the table with the right headers
for table in tables:
    headers = table.find_all('th')
    header_texts = [header.get_text().strip() for header in headers]
    
    # Check if this is the correct table (has Category ID and Description)
    if 'Category ID' in header_texts and 'Description' in header_texts:
        # Get rows from the table (skip header row)
        rows = table.find_all('tr')[1:]
        
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 3:  # Ensure we have at least 3 columns
                category_id = cells[0].get_text().strip()
                description = cells[1].get_text().strip()
                items = cells[2].get_text().strip()
                
                # Extract the link to the category page if available
                link = cells[0].find('a')
                if link:
                    category_link = base_url + link.get('href')
                else:
                    category_link = None
                    
                category_data.append({
                    'Category ID': category_id,
                    'Description': description,
                    'Items': items,
                    'Link': category_link
                })

category_df = pd.DataFrame(category_data)

category_df.head()

,Category ID,Description,Items,Link
0,100032,Reaction time,9,https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id...
1,100029,Numeric memory,15,https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id...
2,100027,Fluid intelligence / reasoning,29,https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id...
3,505,Trail making,8,https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id...
4,501,Matrix pattern completion,4,https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id...


In [8]:
if category_df.empty:
    print("No categories found.")
else:
    print(f"Found {len(category_df)} categories.")
    
    # Initialize final output table
    output_table = []
    
    # For each category, extract the field table
    for idx, row in category_df.iterrows():
        category_id = row['Category ID']
        category_desc = row['Description']
        category_link = row['Link']
        
        print(f"Processing category {category_id}: {category_desc}")
        
        if not category_link:
            continue
            
        print(f"Accessing {category_link}")
        
        try:
            field_response = requests.get(category_link)
            field_response.raise_for_status()
            
            # Parse the HTML
            field_soup = BeautifulSoup(field_response.text, 'html.parser')
            
            # Find all tables
            field_tables = field_soup.find_all('table')
            
            # Look for the table with Field ID and Description
            for field_table in field_tables:
                field_headers = field_table.find_all('th')
                field_header_texts = [header.get_text().strip() for header in field_headers]
                
                # Check if this is the correct table
                if 'Field ID' in field_header_texts and 'Description' in field_header_texts:
                    # Find the indices of the Field ID and Description columns
                    field_id_idx = field_header_texts.index('Field ID')
                    desc_idx = field_header_texts.index('Description')
                    
                    # Get rows from the table (skip header row)
                    field_rows = field_table.find_all('tr')[1:]
                    
                    for field_row in field_rows:
                        field_cells = field_row.find_all('td')
                        if len(field_cells) > max(field_id_idx, desc_idx):
                            field_id = field_cells[field_id_idx].get_text().strip()
                            description = field_cells[desc_idx].get_text().strip()
                            
                            # Clean up field_id - extract just the number if it's a link
                            field_id_clean = re.search(r'(\d+)', field_id)
                            if field_id_clean:
                                field_id = field_id_clean.group(1)
                            
                            output_table.append({
                                'Category ID': category_id,
                                'Category Description': category_desc,
                                'Field ID': field_id,
                                'Field Description': description
                            })
            
            print(f"Found {len(output_table) - len(output_table)} fields for category {category_id}.")
            
        except Exception as e:
            print(f"Error extracting field table for category {category_id}: {e}")
        
        # Add a small delay to be considerate to the website
        time.sleep(1)
    
    # Convert to DataFrame and save
    output_df = pd.DataFrame(output_table)
    

Found 30 categories.
Processing category 100032: Reaction time
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=100032
Found 0 fields for category 100032.
Processing category 100029: Numeric memory
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=100029
Found 0 fields for category 100029.
Processing category 100027: Fluid intelligence / reasoning
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=100027
Found 0 fields for category 100027.
Processing category 505: Trail making
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=505
Found 0 fields for category 505.
Processing category 501: Matrix pattern completion
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=501
Found 0 fields for category 501.
Processing category 502: Symbol digit substitution
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=502
Found 0 fields for category 502.
Processing category 1358: Broken letter recognition
Accessing https://biobank.ndph.ox.ac.uk/ukb/label.cgi?id=1358

In [9]:
output_df

,Category ID,Category Description,Field ID,Field Description
0,100032,Reaction time,20023,Mean time to correctly identify matches
1,100032,Reaction time,401,Index for card A in round
2,100032,Reaction time,10139,Index for card A in round (pilot)
3,100032,Reaction time,402,Index for card B in round
4,100032,Reaction time,10140,Index for card B in round (pilot)
...,...,...,...,...
263,100028,Lights pattern memory,10146,Pattern of lights as remembered (pilot)
264,100028,Lights pattern memory,10144,Time taken to complete lights test (pilot)
265,100077,Word production,10612,Number of words beginning with 'S' (pilot)
266,100077,Word production,10610,Word count (pilot)


In [ ]:
output_df.to_csv("output_table.csv", index=False)
    print(f"Saved {len(output_df)} rows to output_table.csv")